# Image Classification with SciKit-Learn

In this notebook, we'll explore some basic principles for building a classifier for images. Our classifier will be pretty simple, and will differentiate between circles, triangles, and squares.

First let's create some sample images:

In [ ]:
# function to generate an image of random size and color
def create_image (size, shape):
    from random import randint
    import numpy as np
    from PIL import Image, ImageDraw
    
    xy1 = randint(10,40)
    xy2 = randint(60,100)
    col = randint(10,200)

    img = Image.new("RGB", size, (255, 255, 255))
    draw = ImageDraw.Draw(img)
    
    if shape == 'circle':
        draw.ellipse([(xy1,xy1), (xy2,xy2)], fill=col)
    elif shape == 'triangle':
        draw.polygon([(xy1,xy1), (xy2,xy2), (xy2,xy1)], fill=col)
    else:
        draw.rectangle([(xy1,xy1), (xy2,xy2)], fill=col)
    del draw
    
    return np.array(img)

# function to create a dataset of images
def generate_image_data (shapes, size = (128,128), cases = 1000):
    from skimage import exposure
    
    images = []
    imagecodes = []
    
    i = 0
    while(i < cases / len(shapes)):
        for shape in shapes:        
            # Append a new image to collection
            images.append(create_image(size, shape))
            
            # append numeric code for label
            imagecodes.append(shapes.index(shape))
        i = i + 1
    
    return images, imagecodes


# Now we're ready to generate some images
# Our classes will be circles, triangles, and squares
classnames = ['circle', 'triangle', 'square']

# All images will be 128x128 pixels
img_size = (128,128)

# Generate 99 random images.
images, imagecodes = generate_image_data(classnames, img_size, 99)

# Create and display the first three images
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(12, 8))
for i in range(3):
    ax = fig.add_subplot(1, 3, i + 1, xticks=[], yticks=[])
    ax.set_title(classnames[imagecodes[i]])
    ax.imshow(images[i])

## Pre-process the images

Generally, you'll need to clean up image data before training a model. You do this primarily to make the images you'll be training the model with consistent in terms of pixel intensity, contrast, size etc.

In this case, we'll do some basic pre-processing by equalizing the pixel intensity of our images. In reality, you'd also need to resize them images to be a consistent size, apply filters to remove noise, and so on. What we want to end up with is a numpy array that represents the flattened, processed images

*Note: You can ignore the warning produced by the following cell*

In [ ]:
def preprocess_images(image_array):
    from PIL import Image, ImageOps
    
    # Pre-process all of the images to make them consistent
    images = []
    for img in image_array:
        # Equalize the pixel intensity to ensure consistent contrast
        img = ImageOps.equalize(Image.fromarray(img))
        # flatten the images
        images.append(np.array(img).ravel())
        
        # (our images are all the same size - otherwise you should resize them!)
        
    return images

# The pre-processed images are our features, the imagecodes are the labels
import numpy as np
features = np.array(preprocess_images(images))
labels = np.array(imagecodes)
print(labels.size, 'cases ready for training.')

## Split the data
Now that the image data is prepared, we can split it into training and test sets:

In [ ]:
# split into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.30)

print('Training records:',Y_train.size)
print('Test records:',Y_test.size)

## Train the classification model

Now we'll use the prepared image data to train a model.

*(This may take a few minutes)*

In [ ]:
# Train the model
from sklearn.linear_model import LogisticRegression

# Set regularization rate
reg = 0.01

# train a logistic regression model on the training set
clf = LogisticRegression(C=1/reg, solver='lbfgs', multi_class='multinomial').fit(X_train, Y_train)
print (clf)

## Evaluate the model
With our model trained, we'll use it to predict labels for the test data and evaluate its accuracy using the known labels.

In [ ]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

predictions = clf.predict(X_test)
print('Accuracy: ', accuracy_score(Y_test, predictions))

## Use the model with new data
Now we can use the model to classify new images.

In [ ]:
from random import randint
from PIL import Image, ImageOps

# Create a random test image
img = create_image ((128,128), classnames[randint(0, len(classnames)-1)])
plt.imshow(img)

# Modify the image data to match the format of the training features
imgfeatures = np.array(ImageOps.equalize(Image.fromarray(img))).ravel().reshape(1, -1)

pred = clf.predict(imgfeatures)
print('The image is a', classnames[pred[0]])

## Learn More
Take a look at the Image Classification tutorial in the SciKit-Learn documentation at http://scikit-learn.org/stable/tutorial/basic/tutorial.html#introduction